# Business Insights from Active Companies in Belgium

Load the necessary libraries and set up the environment for data analysis.

In [1]:
import sqlite3
import pandas as pd

# Connect to KBO database
conn = sqlite3.connect(r'./data/kbo_database.db')

Run a test query to check the connection to the database and retrieve the first 10 rows from the `enterprises` table.

In [ ]:

query = """
    SELECT *
    FROM enterprise
    LIMIT 5;
"""
df  = pd.read_sql_query(query, conn)
df.head()

,EnterpriseNumber,Status,JuridicalSituation,TypeOfEnterprise,JuridicalForm,JuridicalFormCAC,StartDate
0,0200.065.765,AC,0,2,416.0,None,09-08-1960
1,0200.068.636,AC,0,2,417.0,None,16-02-1923
2,0200.171.970,AC,0,2,116.0,None,01-01-1968
3,0200.245.711,AC,12,2,116.0,None,01-01-1922
4,0200.305.493,AC,0,2,416.0,None,19-03-1962


Get a list of enterprises with some of their details from the `code` table.

In [ ]:
query = """
    SELECT 
        EnterpriseNumber,
        te.Description AS TypeOfEnterprise,
        cs.Description AS Status,
        cjs.Description AS JuridicalSituation,
        cjf.Description AS JuridicalForm,
        cjfcac.Description AS JuridicalFormCAC,
        StartDate
    FROM enterprise AS e
    INNER JOIN code AS te
        ON te.Category = 'TypeOfEnterprise'
        AND te.Language = 'FR'
        AND e.TypeOfEnterprise = te.Code
    INNER JOIN code AS cs
        ON cs.Category = 'Status'
        AND cs.Language = 'FR'
        AND e.Status = cs.Code
    INNER JOIN code AS cjs
        ON cjs.Category = 'JuridicalSituation'
        AND cjs.Language = 'FR'
        AND e.JuridicalSituation = cjs.Code
    INNER JOIN code AS cjf
        ON cjf.Category = 'JuridicalForm'
        AND cjf.Language = 'FR'
        AND e.JuridicalForm = cjf.Code	
    INNER JOIN code AS cjfcac
        ON cjfcac.Category = 'JuridicalForm'
        AND cjfcac.Language = 'FR'
        AND e.JuridicalFormCAC = cjfcac.Code	
    LIMIT 10;
"""

df  = pd.read_sql_query(query, conn)
df.head()

,EnterpriseNumber,TypeOfEnterprise,Status,JuridicalSituation,JuridicalForm,JuridicalFormCAC,StartDate
0,0400.000.680,Personne morale,Actif,Situation normale,Société privée à responsabilité limitée,Société à responsabilité limitée,28-06-1968
1,0400.001.373,Personne morale,Actif,Dissolution volontaire – liquidation,Société privée à responsabilité limitée,Société à responsabilité limitée,01-10-1968
2,0400.004.640,Personne morale,Actif,Dissolution volontaire – liquidation,Société privée à responsabilité limitée,Société à responsabilité limitée,12-10-1968
3,0400.006.026,Personne morale,Actif,Dissolution volontaire – liquidation,Société privée à responsabilité limitée,Société à responsabilité limitée,01-10-1966
4,0400.009.093,Personne morale,Actif,Réunion des parts en une seule main,Société privée à responsabilité limitée,Société à responsabilité limitée,04-10-1968


## Which percentage of companies are under which juridical situation?

In [13]:
query = """
    SELECT 
        cjs.Description AS JuridicalSituation,
        COUNT(*) AS count,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM enterprise), 2) AS percentage
    FROM enterprise AS e
    INNER JOIN code AS cjs
        ON cjs.Category = 'JuridicalSituation'
        AND cjs.Language = 'FR'
        AND e.JuridicalSituation = cjs.Code
    GROUP BY JuridicalSituation
    ORDER BY count DESC;
"""

df  = pd.read_sql_query(query, conn)
df.head(20)

,JuridicalSituation,count,percentage
0,Situation normale,1862383,96.68
1,Ouverture de faillite,32386,1.68
2,Dissolution volontaire – liquidation,25070,1.30
3,Dissolution judiciaire ou nullité,3506,0.18
4,Réunion des parts en une seule main,1457,0.08
5,Dissolution de plein droit,578,0.03
6,Concordat judiciaire avant faillite,423,0.02
7,Sursis (réorganisation judiciaire),250,0.01
8,Ouverture de faillite avec excusabilité,41,0.00
9,Réouverture de liquidation,39,0.00


## Which percentage of companies are under which juridical form?

In [15]:
query = """
    SELECT 
        cjf.Description AS JuridicalForm,
        COUNT(*) AS count,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM enterprise), 3) AS percentage
    FROM enterprise AS e
    INNER JOIN code AS cjf
        ON cjf.Category = 'JuridicalForm'
        AND cjf.Language = 'FR'
        AND e.JuridicalForm = cjf.Code	
    GROUP BY JuridicalForm
    ORDER BY count DESC;
"""

df  = pd.read_sql_query(query, conn)
df.head(110)

,JuridicalForm,count,percentage
0,Société à responsabilité limitée,493384,25.614
1,Association sans but lucratif,139213,7.227
2,Association des copropriétaires,108445,5.630
3,Société privée à responsabilité limitée,86541,4.493
4,Société anonyme,80675,4.188
5,Entité étrangère,67576,3.508
6,Société en commandite,47352,2.458
7,Société ou association sans personnalité jurid...,27737,1.440
8,Société en nom collectif,24888,1.292
9,Société en commandite simple,16182,0.840


## What is the distribution of company statuses?

In [16]:
query = """
    SELECT 
        cs.Description AS Status,
        COUNT(*) AS count,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM enterprise), 2) AS percentage
    FROM enterprise AS e
    INNER JOIN code AS cs
        ON cs.Category = 'Status'
        AND cs.Language = 'FR'
        AND e.Status = cs.Code
    GROUP BY Status
    ORDER BY count DESC;
"""

df  = pd.read_sql_query(query, conn)
df.head()

,Status,count,percentage
0,Actif,1926246,100.0


## Calculate average company age by sector (NACE codes)

In [ ]:
query = """
"""

df  = pd.read_sql_query(query, conn)
df.head()

## Track company creation trends over time with time-based analysis

In [11]:
query = """
    SELECT 
	    CAST(SUBSTR(StartDate, 7, 4) AS INTEGER) AS year,
	    COUNT(*) AS NewCompanies
    FROM enterprise
    WHERE StartDate IS NOT NULL
    GROUP BY year
    ORDER BY year DESC;
"""

df  = pd.read_sql_query(query, conn)
df.head(10)

,year,NewCompanies
0,2025,74092
1,2024,130923
2,2023,115432
3,2022,105626
4,2021,100446
5,2020,84786
6,2019,81171
7,2018,73309
8,2017,66413
9,2016,61028


## Compare geographical distribution of companies

In [ ]:
query = """
"""

df  = pd.read_sql_query(query, conn)
df.head()

## Find growth trends by sector with year-over-year analysis

In [ ]:
query = """
"""

df  = pd.read_sql_query(query, conn)
df.head()

## Detect seasonal patterns for company creation

In [ ]:
query = """
"""

df  = pd.read_sql_query(query, conn)
df.head()

## Which are the rising sectors per region in the the last 5 years?

And what companies within those sectors have the biggest pressence and the fastest grow? 

In [ ]:
query = """
"""

df  = pd.read_sql_query(query, conn)
df.head()